# `pandas02.ipynb`

## Grouping

In [1]:
import pandas as pd
import numpy as np

data = {
    '주문번호': [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010],
    '고객ID': ['A', 'B', 'A', 'C', 'B', 'A', 'D', 'C', 'B', 'D'],
    '상품카테고리': ['전자제품', '의류', '가구', '전자제품', '의류', '식품', '가구', '식품', '전자제품', '의류'],
    '구매액': [150000, 75000, 220000, 95000, 82000, 45000, 180000, 35000, 120000, 62000],
    '배송지역': ['서울', '부산', '서울', '인천', '서울', '부산', '인천', '서울', '부산', '인천'],
    '할인률': [0.05, 0.1, 0, 0.2, 0.1, 0, 0.05, 0.15, 0.2, 0]
}

df = pd.DataFrame(data)

In [2]:
# 기본 그룹화
df.groupby('고객ID')['구매액'].sum()

# 그룹 객체
id_group = df.groupby('고객ID')

# 그룹 확인(그룹핑 기준열의 원소들)
id_group.groups.keys()

# 특정 그룹 데이터 확인
id_group.get_group('A')

# 다중 컬럼 그룹핑
multi_group = df.groupby(['고객ID', '상품카테고리'])['구매액'].mean() # 참고로 이거 시리즈임. 그룹은 전부 index

# 데이터프레임 변환
multi_group.to_frame()

# 집계함수 여러개 적용
df.groupby('고객ID')['구매액'].agg([
    'sum', 'mean', 'count', 'min', 'max'
])

# n개 col에 m개 집계함수
df.groupby('고객ID').agg({
    '구매액': ['sum', 'mean', 'count'],
    '할인률': ['mean', 'max']
})

# 사용자 정의 집계함수
def discount_amount(price):
    return(price * df.loc[price.index, '할인률']).sum()

df.groupby('고객ID')['구매액'].agg([
    ('총구매액', 'sum'),
    ('평균구매액', 'mean'),
    ('할인총액', discount_amount),
])

,총구매액,평균구매액,할인총액
고객ID,,,
A,415000,138333.333333,7500.0
B,277000,92333.333333,39700.0
C,130000,65000.000000,24250.0
D,242000,121000.000000,9000.0


In [3]:
# 집계 함수 응용
import pandas as pd
import numpy as np

# 샘플 데이터
df = pd.DataFrame({
    '상품ID': ['A001', 'A002', 'A001', 'A003', 'A002', 'A004', 'A003', 'A001', 'A002', 'A004'],
    '판매일자': pd.date_range('2023-01-01', periods=10),
    '판매수량': [5, 3, 7, 2, 4, 6, 3, 8, 5, 4],
    '판매금액': [50000, 30000, 70000, 25000, 40000, 65000, 30000, 80000, 50000, 45000],
    '반품수량': [0, 1, 0, 0, 0, 2, 1, 0, 0, 1],
    '고객평점': [4.5, 3.8, 4.2, 5.0, 4.0, 3.5, 4.2, 4.8, 3.9, 4.1]
})

In [4]:
df.groupby('상품ID').agg({
    '판매수량': ['sum', 'mean', 'count'], #alias 등록 가능
    '판매금액': ['sum', 'mean'],
    '반품수량': ['sum'],
    '고객평점': ['mean']
})

판매수량                    판매금액               반품수량 고객평점
      sum      mean count     sum          mean  sum mean
상품ID                                                     
A001   20  6.666667     3  200000  66666.666667    0  4.5
A002   12  4.000000     3  120000  40000.000000    1  3.9
A003    5  2.500000     2   55000  27500.000000    1  4.6
A004   10  5.000000     2  110000  55000.000000    3  3.8

In [5]:
# 커스텀 함수
# 총 판매수량 대비 반품수량 비율 게산
def return_rate(x): # 그러게 x에 뭐가 들어가나
    total_sold = df.loc[x.index, '판매수량'].sum()
    total_returned = df.loc[x.index, '반품수량'].sum()
    return total_returned / total_sold if total_sold > 0 else 0
# python 더 많이 하면서, 입출력 구조에 대한 이해가 높아질 거임.

df.groupby('상품ID').agg({
    '판매수량': ['sum', 'count'],
    '반품수량': ['sum', return_rate],
})

판매수량       반품수량            
      sum count  sum return_rate
상품ID                            
A001   20     3    0    0.000000
A002   12     3    1    0.083333
A003    5     2    1    0.200000
A004   10     2    3    0.300000

In [6]:
# pandas는 SQL을 넘지 못했다.
# 다행이다!

# 그룹별 순위 및 누적 계산
import pandas as pd
import numpy as np

# 샘플 데이터: 부서별 직원 실적
data = {
    '직원ID': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112],
    '이름': ['김철수', '이영희', '박민수', '정지영', '최동민', '강준호', '윤서연', '임태혁', '한미래', '송지원', '오민지', '홍길동'],
    '부서': ['영업', '개발', '영업', '인사', '개발', '영업', '개발', '인사', '영업', '개발', '영업', '인사'],
    '월별실적': [120, 85, 95, 110, 75, 135, 95, 105, 115, 90, 125, 100],
    '고객평가': [4.5, 3.8, 4.2, 4.7, 3.9, 4.8, 4.1, 4.3, 4.5, 4.0, 4.6, 4.2]
}

df = pd.DataFrame(data)
print("부서별 직원 실적 데이터:")
print(df)

부서별 직원 실적 데이터:
    직원ID   이름  부서  월별실적  고객평가
0    101  김철수  영업   120   4.5
1    102  이영희  개발    85   3.8
2    103  박민수  영업    95   4.2
3    104  정지영  인사   110   4.7
4    105  최동민  개발    75   3.9
5    106  강준호  영업   135   4.8
6    107  윤서연  개발    95   4.1
7    108  임태혁  인사   105   4.3
8    109  한미래  영업   115   4.5
9    110  송지원  개발    90   4.0
10   111  오민지  영업   125   4.6
11   112  홍길동  인사   100   4.2


In [7]:
dept_group = df.groupby('부서')

# 부서별 월별 실적 랭킹 내림차순
df['부서랭킹'] = df.groupby('부서')['월별실적'].rank(method='dense', ascending=False)

In [8]:
# 누적 합계 및 누적 통계
# 부서별 누적 실적 합계

df['누적합계'] = df.groupby('부서')['월별실적'].cumsum() # 누적합 매서드
df['부서별누적최댓값'] = df.groupby('부서')['월별실적'].cummax()

# 그룹별 비율계산
# 부서별 총 실적 대비 개인 실적 비율
df['부서총실적'] = df.groupby('부서')['월별실적'].transform('sum')
# transform : 원본 데이터시리즈의 행수에 맞게d
# 위에 왜 sum() 안 썼지?

df['부서기여도'] = df['월별실적'] / df['부서총실적']

# 복합응용
df['성과점수'] = df['월별실적'] * 0.7 + df['고객평가'] * 0.3 * 20
df['부서순위_성과'] = df.groupby('부서')['성과점수'].rank(method='dense', ascending = False)

def cal_bonus(row):
    base_bonus = row['월별실적'] * 0.1
    rank = row['부서순위_성과']
    if rank == 1:
        return base_bonus * 1.5
    elif rank ==2:
        return base_bonus * 1.3
    elif rank ==3:
        return base_bonus * 1.1
    else:
        return base_bonus

df['성과급'] = df.apply(cal_bonus, axis = 1)
df

,직원ID,이름,부서,월별실적,고객평가,부서랭킹,누적합계,부서별누적최댓값,부서총실적,부서기여도,성과점수,부서순위_성과,성과급
0,101,김철수,영업,120,4.5,3.0,120,120,590,0.203390,111.0,3.0,13.20
1,102,이영희,개발,85,3.8,3.0,85,85,345,0.246377,82.3,3.0,9.35
2,103,박민수,영업,95,4.2,5.0,215,120,590,0.161017,91.7,5.0,9.50
3,104,정지영,인사,110,4.7,1.0,110,110,315,0.349206,105.2,1.0,16.50
4,105,최동민,개발,75,3.9,4.0,160,85,345,0.217391,75.9,4.0,7.50
5,106,강준호,영업,135,4.8,1.0,350,135,590,0.228814,123.3,1.0,20.25
6,107,윤서연,개발,95,4.1,1.0,255,95,345,0.275362,91.1,1.0,14.25
7,108,임태혁,인사,105,4.3,2.0,215,110,315,0.333333,99.3,2.0,13.65
8,109,한미래,영업,115,4.5,4.0,465,135,590,0.194915,107.5,4.0,11.50
9,110,송지원,개발,90,4.0,2.0,345,95,345,0.260870,87.0,2.0,11.70


## 실습 매출 데이터

In [9]:
# 실습: 매출 데이터 그룹별 분석
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 매출 데이터 생성
np.random.seed(42)

# 날짜 생성 (2023년 전체)
dates = pd.date_range('2023-01-01', '2023-12-31')
n_records = 500

data = {
    '주문ID': np.arange(1001, 1001 + n_records),
    '주문일자': np.random.choice(dates, n_records),
    '고객ID': np.random.choice([f'CUST{i:03d}' for i in range(1, 101)], n_records),
    '상품ID': np.random.choice([f'PROD{i:03d}' for i in range(1, 51)], n_records),
    '카테고리': np.random.choice(['전자제품', '의류', '가구', '식품', '화장품', '도서', '스포츠'], n_records),
    '매출액': np.random.randint(10000, 500000, n_records),
    '수량': np.random.randint(1, 10, n_records),
    '지역': np.random.choice(['서울', '부산', '인천', '대구', '광주', '대전', '울산', '경기', '강원'], n_records),
    '결제방법': np.random.choice(['신용카드', '현금', '체크카드', '휴대폰', '계좌이체'], n_records),
    '고객등급': np.random.choice(['일반', '실버', '골드', 'VIP'], n_records)
}

df = pd.DataFrame(data)

df

,주문ID,주문일자,고객ID,상품ID,카테고리,매출액,수량,지역,결제방법,고객등급
0,1001,2023-04-13,CUST002,PROD011,식품,384899,4,대구,현금,골드
1,1002,2023-12-15,CUST092,PROD004,도서,66985,7,대전,체크카드,VIP
2,1003,2023-09-28,CUST032,PROD015,가구,322213,9,강원,체크카드,일반
3,1004,2023-04-17,CUST091,PROD006,스포츠,324470,2,대전,체크카드,실버
4,1005,2023-03-13,CUST084,PROD049,화장품,118737,3,강원,현금,VIP
...,...,...,...,...,...,...,...,...,...,...
495,1496,2023-11-21,CUST095,PROD049,전자제품,44084,2,광주,체크카드,VIP
496,1497,2023-01-04,CUST006,PROD017,화장품,261459,3,경기,체크카드,실버
497,1498,2023-01-16,CUST066,PROD010,도서,120641,1,경기,계좌이체,실버
498,1499,2023-10-07,CUST084,PROD049,전자제품,115111,8,인천,현금,실버


In [ ]:
# 검색 후 하기
# 날짜 정보 추출 -> 컬럼 추가[주문년월, 요일, 주]
df['주문년월'] = df['주문일자'].dt.strftime('%Y-%m')
df['요일'] = df['주문일자'].dt.day_name()
df['주'] = df['주문일자'].dt.isocalendar().week

# 1월 1일인 행만 확인하기
df[df['주문일자'] == '2023-01-01']  # 얘는 52주임.(iso 기준 맞추다보니 쩔수)

,주문ID,주문일자,고객ID,상품ID,카테고리,매출액,수량,지역,결제방법,고객등급,주문년월,요일,주,단가,상품랭킹
446,1447,2023-01-01,CUST090,PROD011,전자제품,267368,6,강원,체크카드,골드,2023-01,Sunday,52,44561.333333,7.0


In [39]:
# 단가 계산(총매출액 / 수량)
df['단가'] = df['매출액'] / df['수량']
# 단가 컬럼 추가
df

,주문ID,주문일자,고객ID,상품ID,카테고리,매출액,수량,지역,결제방법,고객등급,주문년월,요일,주,단가,상품랭킹
0,1001,2023-04-13,CUST002,PROD011,식품,384899,4,대구,현금,골드,2023-04,Thursday,15,96224.750000,2.0
1,1002,2023-12-15,CUST092,PROD004,도서,66985,7,대전,체크카드,VIP,2023-12,Friday,50,9569.285714,6.0
2,1003,2023-09-28,CUST032,PROD015,가구,322213,9,강원,체크카드,일반,2023-09,Thursday,39,35801.444444,2.0
3,1004,2023-04-17,CUST091,PROD006,스포츠,324470,2,대전,체크카드,실버,2023-04,Monday,16,162235.000000,5.0
4,1005,2023-03-13,CUST084,PROD049,화장품,118737,3,강원,현금,VIP,2023-03,Monday,11,39579.000000,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1496,2023-11-21,CUST095,PROD049,전자제품,44084,2,광주,체크카드,VIP,2023-11,Tuesday,47,22042.000000,13.0
496,1497,2023-01-04,CUST006,PROD017,화장품,261459,3,경기,체크카드,실버,2023-01,Wednesday,1,87153.000000,8.0
497,1498,2023-01-16,CUST066,PROD010,도서,120641,1,경기,계좌이체,실버,2023-01,Monday,3,120641.000000,12.0
498,1499,2023-10-07,CUST084,PROD049,전자제품,115111,8,인천,현금,실버,2023-10,Saturday,40,14388.875000,9.0


In [40]:
# 카테고리별 매출 분석(매출의 총합, 평균, 개수), 수량, 총합
df.groupby('카테고리').agg({
    '매출액': [
        ('매출총합', 'sum'),
        ('평균', 'mean'),
        ('개수', 'count')
        ],
    '수량': ['sum']        # 여기가 무슨 말이지?
}).round(2)

매출액                  수량
          매출총합         평균  개수  sum
카테고리                              
가구    19263561  267549.46  72  348
도서    17826814  254668.77  70  374
스포츠   17729088  268622.55  66  332
식품    16816186  254790.70  66  299
의류    17971859  253124.77  71  359
전자제품  19526279  256924.72  76  358
화장품   20600588  260766.94  79  386

In [43]:
# 월별 매출 트렌드
# 주문년월 컬럼으로 매출액 sum, 주문ID count, 단가 mean
df.groupby('주문년월').agg({
    '매출액': ['sum', 'mean', 'count'],
    '주문ID': ['count'],      # what does mean ID???
    '단가': ['mean']
}).rename(columns={'주문ID': '주문건수'}).round(2)

매출액                   주문건수         단가
              sum       mean count count       mean
주문년월                                               
2023-01   7786750  251185.48    31    31   68573.32
2023-02  13558838  288485.91    47    47   83141.03
2023-03   4820044  209567.13    23    23   67224.54
2023-04  13329098  272022.41    49    49   97647.61
2023-05  12825903  251488.29    51    51   88883.44
2023-06  10302473  234147.11    44    44   64922.53
2023-07  11575996  312864.76    37    37  107918.45
2023-08  11604378  246901.66    47    47   69333.94
2023-09  11646019  258800.42    45    45   76389.96
2023-10  10932657  254247.84    43    43   72844.86
2023-11   9335449  245669.71    38    38   76846.46
2023-12  12016770  267039.33    45    45   82730.37

In [45]:
# 지역 & 카테고리별 매출 분석
df.groupby(['지역', '카테고리']).agg({
    '매출액': [
        ('매출총액', 'sum'),
        ('매출평균', 'mean'),
        ('매출건수', 'count')
    ]
}).round(2)

매출액                
            매출총액       매출평균 매출건수
지역 카테고리                         
강원 가구    2567149  233377.18   11
   도서    2229004  371500.67    6
   스포츠   3236879  269739.92   12
   식품    1546551  257758.50    6
   의류    1961318  280188.29    7
...          ...        ...  ...
인천 스포츠   1677507  279584.50    6
   식품    2226374  202397.64   11
   의류    2061788  206178.80   10
   전자제품  2870028  287002.80   10
   화장품   2365803  295725.38    8

[63 rows x 3 columns]

In [ ]:
# Unstack과 히트맵 시각화
regional_df = df.groupby(['지역', '카테고리','결제방법', '주문년월'])['매출액'].sum().unstack()
# Unstack: 다중인덱스 구조를 column으로 펼처서 재구조화함.
# 맨 마지막 col을 위로 올리는 구조인 듯.
regional_df

주문년월          2023-01   2023-02  2023-03   2023-04   2023-05   2023-06  \
지역 카테고리 결제방법                                                             
강원 가구   계좌이체      NaN       NaN      NaN       NaN  326205.0       NaN   
        신용카드      NaN       NaN      NaN       NaN       NaN  152635.0   
        체크카드      NaN  418529.0      NaN  138803.0       NaN       NaN   
        현금        NaN       NaN      NaN  111459.0       NaN       NaN   
        휴대폰       NaN       NaN      NaN       NaN       NaN  176703.0   
...               ...       ...      ...       ...       ...       ...   
인천 전자제품 휴대폰       NaN       NaN      NaN       NaN       NaN       NaN   
   화장품  계좌이체      NaN       NaN  81635.0       NaN       NaN       NaN   
        신용카드      NaN       NaN      NaN       NaN       NaN       NaN   
        체크카드      NaN       NaN      NaN       NaN       NaN       NaN   
        현금        NaN       NaN      NaN       NaN       NaN       NaN   

주문년월           2023-07   2023-08   2023-09   2023-10   2023-11   2023-12  
지역 카테고리 결제방법                                                              
강원 가구   계좌이체       NaN       NaN       NaN       NaN       NaN       NaN  
        신용카드       NaN       NaN       NaN       NaN       NaN       NaN  
        체크카드       NaN       NaN  322213.0       NaN       NaN       NaN  
        현금         NaN       NaN       NaN  361537.0  141691.0       NaN  
        휴대폰   255014.0       NaN       NaN       NaN       NaN  162360.0  
...                ...       ...       ...       ...       ...       ...  
인천 전자제품 휴대폰        NaN       NaN  473183.0       NaN  233131.0       NaN  
   화장품  계좌이체       NaN  357022.0       NaN  231097.0       NaN       NaN  
        신용카드       NaN       NaN  404583.0       NaN  409492.0       NaN  
        체크카드       NaN       NaN  130030.0       NaN       NaN       NaN  
        현금         NaN       NaN       NaN       NaN       NaN  751944.0  

[259 rows x 12 columns]

In [50]:
# 요일 & 고객등급별 매출 패턴 분석
df.groupby(['요일', '고객등급']).agg({
    '매출액': ['sum', 'mean', 'count']
}).round(2)

매출액                 
                    sum       mean count
요일        고객등급                          
Friday    VIP   2998646  230665.08    13
          골드    5563601  309088.94    18
          실버    4584163  286510.19    16
          일반    4059550  225530.56    18
Monday    VIP   6182434  281019.73    22
          골드    3402384  226825.60    15
          실버    4532290  238541.58    19
          일반    7265467  269091.37    27
Saturday  VIP   4298206  252835.65    17
          골드    4124191  274946.07    15
          실버    5460233  273011.65    20
          일반    4178794  261174.62    16
Sunday    VIP   4838696  254668.21    19
          골드    3240409  202525.56    16
          실버    4501861  281366.31    16
          일반    5079060  241860.00    21
Thursday  VIP   5066444  253322.20    20
          골드    5601373  294809.11    19
          실버    4273034  284868.93    15
          일반    6456148  258245.92    25
Tuesday   VIP   4955018  275278.78    18
          골드    5357448  255116.57    21
          실버    4777296  318486.40    15
          일반    4184996  246176.24    17
Wednesday VIP   3323716  221581.07    15
          골드    4295608  204552.76    21
          실버    3749879  312489.92    12
          일반    3383430  241673.57    14

In [59]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['요일'] = pd.Categorical(df['요일'], categories=day_order, ordered=True)
df.groupby(['고객등급', '요일'])['매출액'].sum().unstack()

/var/folders/tx/t7px7cn503bgk6zlg3m0b4740000gn/T/ipykernel_67155/3750161729.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['고객등급', '요일'])['매출액'].sum().unstack()


요일,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
고객등급,,,,,,,
VIP,6182434,4955018,3323716,5066444,2998646,4298206,4838696
골드,3402384,5357448,4295608,5601373,5563601,4124191,3240409
실버,4532290,4777296,3749879,4273034,4584163,5460233,4501861
일반,7265467,4184996,3383430,6456148,4059550,4178794,5079060


In [ ]:
# 결제방법별 분석 및 고객 행동(매출액의 sum, mean, count / 단가 mean)
# My 답
df.groupby('결제방법').agg({
    '매출액': ['sum', 'mean', 'count'],
    '단가': 'mean'
}).round(2)

매출액                         단가
           sum       mean count      mean
결제방법                                     
계좌이체  26376866  251208.25   105  81029.84
신용카드  25514730  260354.39    98  89773.78
체크카드  26536652  260163.25   102  66063.70
현금    25579043  272117.48    94  81782.44
휴대폰   25727084  254723.60   101  84024.98

In [67]:
# 비율 따는 법

total_sales = df['매출액'].sum()
# 이렇게 해야 계산이 좀 덜해지겠지???

def ratio(x):
    return x.sum() * 100 / total_sales

df.groupby('결제방법').agg({
    '매출액': ['sum', 'mean', 'count', 
            ('비율(%)', ratio)
            ],
    '단가': 'mean'
}).round(3)

매출액                                   단가
           sum        mean count   비율(%)       mean
결제방법                                               
계좌이체  26376866  251208.248   105  20.331  81029.835
신용카드  25514730  260354.388    98  19.667  89773.785
체크카드  26536652  260163.255   102  20.455  66063.698
현금    25579043  272117.479    94  19.716  81782.443
휴대폰   25727084  254723.604   101  19.831  84024.975

In [73]:
# 고객별 구매패턴(매출액 sum mena count, 상품 고유 수, 카테고리 고유 수)
df.groupby('고객ID').agg({
    '매출액': ['sum', 'mean', 'count'],
    '상품ID': lambda x: x.nunique(),
    '상품ID':  'nunique',
    '카테고리': 'nunique'
}).round(2).sort_values(('매출액', 'sum'), ascending=False).head(3)

# sort_values(['a', 'b']) => ORDER BY a, b(이중 기준)
# sort_values(('a', 'sum')) => col 'a'의 'sum' 값을 기준으로 정렬
# sort_values([('a', 'sum'), 'b']) => 이거 이해하면 됨 ㅇㅇ

매출액                     상품ID    카테고리
             sum       mean count nunique nunique
고객ID                                             
CUST092  2814587  255871.55    11      11       5
CUST017  2814138  281413.80    10      10       6
CUST005  2709222  338652.75     8       8       5

In [ ]:
# 주별 매출 추이
weekly_sales = df.groupby('주').agg({
    '매출액': ['sum', 'mean', 'count'],
}).round(2)

매출액                 
        sum       mean count
주                           
1   1693663  211707.88     8
2   1796630  224578.75     8
3   1796731  224591.38     8
4   2232358  372059.67     6
5   2203467  314781.00     7
6   2250272  250030.22     9
7   3313923  301265.73    11
8   4689983  312665.53    15
9   2898753  241562.75    12
10  1507064  301412.80     5
11   534285  178095.00     3
12   405797  135265.67     3
13  1202273  150284.12     8
14  4330412  288694.13    15
15  3668608  282200.62    13
16  2869527  286952.70    10
17  1833616  229202.00     8
18  2462466  223860.55    11
19  4196672  299762.29    14
20  1560857  222979.57     7
21  3616903  241126.87    15
22  2092128  190193.45    11
23  3771165  290089.62    13
24  1288969  257793.80     5
25  3454539  246752.79    14
26   684677  136935.40     5
27  4467802  297853.47    15
28  2382578  297822.25     8
29  3291665  365740.56     9
30  1433951  286790.20     5
31  3730343  339122.09    11
32  2286155  207832.27    11
33  2467888  274209.78     9
34  2011473  167622.75    12
35  1325247  189321.00     7
36  2894343  263122.09    11
37  3375886  281323.83    12
38  2242428  249158.67     9
39  3085667  280515.18    11
40  2869975  239164.58    12
41  1901637  172876.09    11
42  3201310  355701.11     9
43  1574426  314885.20     5
44  2698092  224841.00    12
45  2066284  229587.11     9
46  2998060  299806.00    10
47  2454792  245479.20    10
48   793539  264513.00     3
49  3308152  254473.23    13
50  3354545  223636.33    15
51  1593366  265561.00     6
52  3569033  324457.55    11

In [80]:
# 매출 TOP 10 상품
df['상품랭킹'] = df.groupby('상품ID')['매출액'].rank(method='dense', ascending = False)
df

df.groupby('상품ID')['매출액'].sum().sort_values(ascending=False).head(10)

상품ID
PROD040    4878382
PROD029    4260616
PROD038    3825127
PROD047    3793126
PROD009    3737512
PROD033    3645223
PROD010    3625484
PROD017    3608202
PROD036    3543357
PROD026    3516412
Name: 매출액, dtype: int64

## 데이터 결합
1. 데이터 단순 결합`concat`
2. 데이터 병합`merge`
3. Index 기준 `Join`

In [86]:
# 데이터 결합 기초 (concat) - 행결합 (Col 똑같아야 함)

# 샘플 데이터 생성
# 첫 번째 데이터프레임: 1월 판매 데이터
df1 = pd.DataFrame({
    '상품ID': ['A001', 'A002', 'A003', 'A004', 'A005'],
    '상품명': ['노트북', '스마트폰', '태블릿', '헤드폰', '스피커'],
    '판매량_1월': [10, 20, 15, 30, 25]
})

# 두 번째 데이터프레임: 2월 판매 데이터
df2 = pd.DataFrame({
    '상품ID': ['A001', 'A003', 'A005', 'A006', 'A007'],
    '상품명': ['노트북', '태블릿', '스피커', '마우스', '키보드'],
    '판매량_2월': [12, 18, 23, 15, 19]
})

# 기본 concat - 행 결합 -> 안맞는 컬럼은 NaN
pd.concat([df1, df2])

# 인덱스 초기화 (겹치는 인덱스 없이 처음부터 다시)
pd.concat([df1, df2], ignore_index=True)

# 열 방향 결합
pd.concat([df1, df2], axis=1)

# join inner (공통 열만 유지)
pd.concat([df1, df2], join='inner')

,상품ID,상품명
0,A001,노트북
1,A002,스마트폰
2,A003,태블릿
3,A004,헤드폰
4,A005,스피커
0,A001,노트북
1,A003,태블릿
2,A005,스피커
3,A006,마우스
4,A007,키보드


In [ ]:
# 데이터 병합 (merge)
import pandas as pd
import numpy as np

# 샘플 데이터 생성
# 상품 정보 데이터프레임
products = pd.DataFrame({
    '상품ID': ['P001', 'P002', 'P003', 'P004', 'P005'],
    '상품명': ['노트북', '스마트폰', '태블릿', '헤드폰', '스피커'],
    '가격': [1200000, 850000, 500000, 150000, 75000],
    '카테고리': ['컴퓨터', '모바일', '모바일', '음향기기', '음향기기']
})

# 주문 정보 데이터프레임
orders = pd.DataFrame({
    '주문번호': [1001, 1002, 1003, 1004, 1005, 1006],
    '고객ID': ['C001', 'C002', 'C003', 'C001', 'C004', 'C002'],
    '상품ID': ['P001', 'P002', 'P003', 'P002', 'P005', 'P006'],
    '수량': [1, 2, 1, 1, 3, 2],
    '주문일자': ['2023-01-05', '2023-01-10', '2023-01-15', '2023-01-20', '2023-01-25', '2023-01-30']
})

print("상품 정보:")
print(products)
print("\n주문 정보:")
print(orders)

# 기본병합(Inner Join)
pd.merge(orders, products, on='상품ID') # 상품ID 기준 Inner Join

# Outer Join
pd.merge(orders, products, on='상품ID', how='outer')

# Left Join
pd.merge(orders, products, on='상품ID', how='left')



상품 정보:
   상품ID   상품명       가격  카테고리
0  P001   노트북  1200000   컴퓨터
1  P002  스마트폰   850000   모바일
2  P003   태블릿   500000   모바일
3  P004   헤드폰   150000  음향기기
4  P005   스피커    75000  음향기기

주문 정보:
   주문번호  고객ID  상품ID  수량        주문일자
0  1001  C001  P001   1  2023-01-05
1  1002  C002  P002   2  2023-01-10
2  1003  C003  P003   1  2023-01-15
3  1004  C001  P002   1  2023-01-20
4  1005  C004  P005   3  2023-01-25
5  1006  C002  P006   2  2023-01-30


,주문번호,고객ID,상품ID,수량,주문일자,상품명,가격,카테고리
0,1001,C001,P001,1,2023-01-05,노트북,1200000.0,컴퓨터
1,1002,C002,P002,2,2023-01-10,스마트폰,850000.0,모바일
2,1003,C003,P003,1,2023-01-15,태블릿,500000.0,모바일
3,1004,C001,P002,1,2023-01-20,스마트폰,850000.0,모바일
4,1005,C004,P005,3,2023-01-25,스피커,75000.0,음향기기
5,1006,C002,P006,2,2023-01-30,NaN,NaN,NaN


In [87]:
customers = pd.DataFrame({
    'ID': ['C001', 'C002', 'C003', 'C004', 'C005'],
    '이름': ['김철수', '이영희', '박민수', '정지영', '최동민'],
    '등급': ['VIP', '골드', '실버', '골드', '브론즈']
})

# 열 이름이 다르면?
pd.merge(
    orders, 
    customers, 
    left_on='고객ID',  # orders 데이터프레임의 열 이름
    right_on='ID',    # customers 데이터프레임의 열 이름
    how='inner'
)

,주문번호,고객ID,상품ID,수량,주문일자,ID,이름,등급
0,1001,C001,P001,1,2023-01-05,C001,김철수,VIP
1,1002,C002,P002,2,2023-01-10,C002,이영희,골드
2,1003,C003,P003,1,2023-01-15,C003,박민수,실버
3,1004,C001,P002,1,2023-01-20,C001,김철수,VIP
4,1005,C004,P005,3,2023-01-25,C004,정지영,골드
5,1006,C002,P006,2,2023-01-30,C002,이영희,골드


In [2]:
import numpy as np
import pandas as pd


# 샘플 데이터셋 생성
# 1. 고객 정보 데이터
customers = pd.DataFrame({
    '고객ID': [f'CUST{i:03d}' for i in range(1, 11)],
    '이름': ['김철수', '이영희', '박민수', '정지영', '최동민', '강준호', '윤서연', '임태혁', '한미래', '송지원'],
    '성별': ['남', '여', '남', '여', '남', '남', '여', '남', '여', '여'],
    '연령대': ['30대', '20대', '40대', '30대', '50대', '20대', '40대', '30대', '20대', '50대'],
    '가입일자': pd.date_range('2023-01-01', periods=10, freq='3D'),
    '지역': ['서울', '부산', '서울', '인천', '대구', '서울', '부산', '인천', '서울', '대구']
})

# 2. 주문 정보 데이터
np.random.seed(42)
n_orders = 50

orders = pd.DataFrame({
    '주문번호': [f'ORD{i:04d}' for i in range(1, n_orders+1)],
    '고객ID': np.random.choice(customers['고객ID'], n_orders),
    '주문일자': pd.date_range('2023-01-05', periods=n_orders, freq='2D'),
    '결제방법': np.random.choice(['신용카드', '체크카드', '계좌이체', '간편결제'], n_orders),
    '배송상태': np.random.choice(['배송완료', '배송중', '주문확인', '배송지연'], n_orders, p=[0.7, 0.15, 0.1, 0.05])
})

# 3. 주문 상세 정보 데이터
n_details = 80
products = ['노트북', '스마트폰', '태블릿', '헤드폰', '스피커', '키보드', '마우스', '모니터']
categories = ['전자제품', '컴퓨터', '주변기기', '음향기기']

order_details = pd.DataFrame({
    '상세번호': [f'ITEM{i:04d}' for i in range(1, n_details+1)],
    '주문번호': np.random.choice(orders['주문번호'], n_details),
    '상품명': np.random.choice(products, n_details),
    '카테고리': np.random.choice(categories, n_details),
    '수량': np.random.randint(1, 5, n_details),
    '가격': np.random.choice([50000, 100000, 150000, 800000, 1200000, 1500000], n_details),
    '할인율': np.random.choice([0, 0.1, 0.2, 0.3], n_details)
})

# 4. 배송 정보 데이터
shipping = pd.DataFrame({
    '주문번호': orders['주문번호'].unique(),
    '배송사': np.random.choice(['A택배', 'B물류', 'C익스프레스'], len(orders['주문번호'].unique())),
    '배송비': np.random.choice([0, 2500, 5000], len(orders['주문번호'].unique())),
    '출고일자': pd.date_range('2023-01-06', periods=len(orders['주문번호'].unique()), freq='2D')
})

# 5. 고객 만족도 데이터 (일부 주문에 대해서만)
satisfaction_orders = np.random.choice(orders['주문번호'], size=30, replace=False)
satisfaction = pd.DataFrame({
    '주문번호': satisfaction_orders,
    '만족도': np.random.randint(1, 6, 30),
    '리뷰': np.random.choice(['긍정', '중립', '부정'], 30, p=[0.6, 0.3, 0.1]),
    '리뷰일자': pd.date_range('2023-01-15', periods=30, freq='3D')
})

In [5]:
order_details.head(10)

,상세번호,주문번호,상품명,카테고리,수량,가격,할인율
0,ITEM0001,ORD0044,마우스,음향기기,2,1200000,0.2
1,ITEM0002,ORD0020,헤드폰,전자제품,1,50000,0.1
2,ITEM0003,ORD0030,노트북,컴퓨터,2,1200000,0.1
3,ITEM0004,ORD0011,노트북,전자제품,3,800000,0.2
4,ITEM0005,ORD0028,모니터,음향기기,2,800000,0.1
5,ITEM0006,ORD0025,모니터,음향기기,2,800000,0.3
6,ITEM0007,ORD0039,마우스,음향기기,3,1200000,0.0
7,ITEM0008,ORD0033,스마트폰,음향기기,4,800000,0.3
8,ITEM0009,ORD0001,모니터,음향기기,2,1500000,0.0
9,ITEM0010,ORD0027,노트북,주변기기,4,1200000,0.0


In [ ]:
# 주문 + 주문상세
# left 쓰게 되면 앞에 있는 거 기준으로 합쳐짐
pd.merge(orders, order_details, on='주문번호', how='inner').head()
# 주문번호는 겹치는 거니까 하나만 써져있음.

,주문번호,고객ID,주문일자,결제방법,배송상태,상세번호,상품명,카테고리,수량,가격,할인율
0,ORD0001,CUST007,2023-01-05,계좌이체,배송중,ITEM0009,모니터,음향기기,2,1500000,0.0
1,ORD0001,CUST007,2023-01-05,계좌이체,배송중,ITEM0047,헤드폰,음향기기,4,1200000,0.1
2,ORD0002,CUST004,2023-01-07,계좌이체,배송완료,ITEM0033,키보드,전자제품,2,50000,0.0
3,ORD0002,CUST004,2023-01-07,계좌이체,배송완료,ITEM0040,노트북,전자제품,2,50000,0.2
4,ORD0002,CUST004,2023-01-07,계좌이체,배송완료,ITEM0041,노트북,음향기기,3,1200000,0.3


In [11]:
# 상동.
# 주문에 '주문상세'를 합친다.(left 아님)
orders.merge(order_details, on='주문번호', how='inner').head()

,주문번호,고객ID,주문일자,결제방법,배송상태,상세번호,상품명,카테고리,수량,가격,할인율
0,ORD0001,CUST007,2023-01-05,계좌이체,배송중,ITEM0009,모니터,음향기기,2,1500000,0.0
1,ORD0001,CUST007,2023-01-05,계좌이체,배송중,ITEM0047,헤드폰,음향기기,4,1200000,0.1
2,ORD0002,CUST004,2023-01-07,계좌이체,배송완료,ITEM0033,키보드,전자제품,2,50000,0.0
3,ORD0002,CUST004,2023-01-07,계좌이체,배송완료,ITEM0040,노트북,전자제품,2,50000,0.2
4,ORD0002,CUST004,2023-01-07,계좌이체,배송완료,ITEM0041,노트북,음향기기,3,1200000,0.3


### # ***예제***: `order_details`에서 `총금액` 뽑아서, `orders`랑 합치기.

In [37]:


# 1. 총 주문금액 계산
total_series = order_details['수량'] * order_details['가격'] * (1 - order_details['할인율'])
# 꿀팁 하나. 'total = ' 먼저 쓰지 말고 뒤에 거 먼저 쓰면서 변수 하나씩 살펴보면서 코딩하기.
total_series    # 자료형 정보 변수명에 써두기.
order_details['총금액'] = total_series
order_details

# SQL이랑 다른 점? SQL은 컬럼 추가하는 게 꽤 어렵다.(사실 DA는 컬럼 추가하는 못된 짓을 하면 안 됨)

# 2. 주문별 합계 계산
order_totals = order_details.groupby('주문번호')['총금액'].sum().reset_index()
order_totals.head()
# reset_index: 원래 Index를 컬럼화시켜버림(인덱스는 0 ~ n)

orders_with_totals = orders.merge(order_totals, on='주문번호', how='left')
orders_with_totals.head()

,주문번호,고객ID,주문일자,결제방법,배송상태,총금액
0,ORD0001,CUST007,2023-01-05,계좌이체,배송중,7320000.0
1,ORD0002,CUST004,2023-01-07,계좌이체,배송완료,3980000.0
2,ORD0003,CUST008,2023-01-09,체크카드,배송지연,6225000.0
3,ORD0004,CUST005,2023-01-11,신용카드,배송중,2520000.0
4,ORD0005,CUST007,2023-01-13,간편결제,배송완료,4050000.0


In [36]:
# 중복된 데이터를 모아서 보기
order_details[order_details.duplicated(subset=['주문번호'], keep=False)].sort_values('주문번호')

,상세번호,주문번호,상품명,카테고리,수량,가격,할인율,총금액
46,ITEM0047,ORD0001,헤드폰,음향기기,4,1200000,0.1,4320000.0
8,ITEM0009,ORD0001,모니터,음향기기,2,1500000,0.0,3000000.0
32,ITEM0033,ORD0002,키보드,전자제품,2,50000,0.0,100000.0
39,ITEM0040,ORD0002,노트북,전자제품,2,50000,0.2,80000.0
40,ITEM0041,ORD0002,노트북,음향기기,3,1200000,0.3,2520000.0
...,...,...,...,...,...,...,...,...
36,ITEM0037,ORD0049,모니터,주변기기,4,100000,0.0,400000.0
34,ITEM0035,ORD0049,스피커,전자제품,2,100000,0.2,160000.0
29,ITEM0030,ORD0049,스마트폰,주변기기,1,50000,0.0,50000.0
28,ITEM0029,ORD0049,노트북,음향기기,3,1500000,0.0,4500000.0


In [42]:
# 주문정보에 고객정보 결합
orders_customers = orders_with_totals.merge(customers, on='고객ID') #, how='left')
orders_customers.head()
# 결측치 어떡하징
    # 1) NaN 0으로 만들기
    # 2) 저 인간들 리스트를 만들어서 탐사를 조진다
    # 3) (분야를 나눈)평균값으로 채운다.(현실 데이터랑 가깝게 만들기~) 그런 걸 꼭 해야되나...?

,주문번호,고객ID,주문일자,결제방법,배송상태,총금액,이름,성별,연령대,가입일자,지역
0,ORD0001,CUST007,2023-01-05,계좌이체,배송중,7320000.0,윤서연,여,40대,2023-01-19,부산
1,ORD0002,CUST004,2023-01-07,계좌이체,배송완료,3980000.0,정지영,여,30대,2023-01-10,인천
2,ORD0003,CUST008,2023-01-09,체크카드,배송지연,6225000.0,임태혁,남,30대,2023-01-22,인천
3,ORD0004,CUST005,2023-01-11,신용카드,배송중,2520000.0,최동민,남,50대,2023-01-13,대구
4,ORD0005,CUST007,2023-01-13,간편결제,배송완료,4050000.0,윤서연,여,40대,2023-01-19,부산


In [51]:
# 배송정보 추가
# DF를 점점 늘려나가는 느낌으로다가/
full_orders = orders_customers.merge(shipping, on='주문번호')
full_orders.head()

# 배송 소요일 추가
full_orders['배송소요일'] = (full_orders['출고일자'] - full_orders['주문일자']).dt.days
full_orders.head()

,주문번호,고객ID,주문일자,결제방법,배송상태,총금액,이름,성별,연령대,가입일자,지역,배송사,배송비,출고일자,배송소요일
0,ORD0001,CUST007,2023-01-05,계좌이체,배송중,7320000.0,윤서연,여,40대,2023-01-19,부산,A택배,2500,2023-01-06,1
1,ORD0002,CUST004,2023-01-07,계좌이체,배송완료,3980000.0,정지영,여,30대,2023-01-10,인천,C익스프레스,5000,2023-01-08,1
2,ORD0003,CUST008,2023-01-09,체크카드,배송지연,6225000.0,임태혁,남,30대,2023-01-22,인천,A택배,2500,2023-01-10,1
3,ORD0004,CUST005,2023-01-11,신용카드,배송중,2520000.0,최동민,남,50대,2023-01-13,대구,C익스프레스,5000,2023-01-12,1
4,ORD0005,CUST007,2023-01-13,간편결제,배송완료,4050000.0,윤서연,여,40대,2023-01-19,부산,B물류,2500,2023-01-14,1


In [57]:
# 만족도 정보 합치기
order_with_satisf = full_orders.merge(satisfaction, on='주문번호', how='left')
order_with_satisf.head()
# 만족도리뷰 있는 게 30개, 전체는 50개. 그러므로 left 사용.
# NaN: Not a Number / NaT: Not a Time

,주문번호,고객ID,주문일자,결제방법,배송상태,총금액,이름,성별,연령대,가입일자,지역,배송사,배송비,출고일자,배송소요일,만족도,리뷰,리뷰일자
0,ORD0001,CUST007,2023-01-05,계좌이체,배송중,7320000.0,윤서연,여,40대,2023-01-19,부산,A택배,2500,2023-01-06,1,NaN,NaN,NaT
1,ORD0002,CUST004,2023-01-07,계좌이체,배송완료,3980000.0,정지영,여,30대,2023-01-10,인천,C익스프레스,5000,2023-01-08,1,5.0,부정,2023-01-21
2,ORD0003,CUST008,2023-01-09,체크카드,배송지연,6225000.0,임태혁,남,30대,2023-01-22,인천,A택배,2500,2023-01-10,1,NaN,NaN,NaT
3,ORD0004,CUST005,2023-01-11,신용카드,배송중,2520000.0,최동민,남,50대,2023-01-13,대구,C익스프레스,5000,2023-01-12,1,4.0,중립,2023-03-16
4,ORD0005,CUST007,2023-01-13,간편결제,배송완료,4050000.0,윤서연,여,40대,2023-01-19,부산,B물류,2500,2023-01-14,1,5.0,부정,2023-03-19


In [68]:
# 분석: 연령대별 성별 총 주문금액
spending_stat = order_with_satisf.groupby(['연령대', '성별'])['총금액'].sum().unstack()
spending_stat

성별,남,여
연령대,,
20대,4550000.0,26820000.0
30대,15555000.0,16140000.0
40대,20675000.0,27220000.0
50대,7270000.0,18780000.0


In [71]:
# 분석: 결제방법별 평균 주문금액 및 건수
how_to_pay = order_with_satisf.groupby('결제방법').agg({
    '총금액': ['mean', 'sum', 'count']
})

how_to_pay

총금액                  
              mean         sum count
결제방법                                
간편결제  3.911111e+06  35200000.0     9
계좌이체  3.934000e+06  19670000.0     5
신용카드  3.605000e+06  32445000.0     9
체크카드  3.549643e+06  49695000.0    14

In [82]:
# 만족도와 총 주문 금액 관계

# 일단 만족도 없는 애들 지워.(결측치 제거)
ord_stf = order_with_satisf.dropna(subset=['리뷰'])

# 만족도별 평균 주문 금액
stf_spend = ord_stf.groupby('만족도')['총금액'].mean().reset_index()
stf_spend

,만족도,총금액
0,1.0,4.000000e+05
1,2.0,7.275000e+05
2,3.0,1.266667e+06
3,4.0,5.803333e+06
4,5.0,3.610000e+06


In [93]:
# 지역별 배송사 선호
rigion_ship = order_with_satisf.groupby(['지역', '배송사']).size().unstack().fillna(0)
# 이렇게 쓸 수도 있음: order_with_satisf.groupby(['지역', '배송사'])['주문번호'].count()


# pandas의 장점 : 컬럼을 추가해서 테이블을 편하게 편집할 수 있기 때문.
rigion_ship

배송사,A택배,B물류,C익스프레스
지역,,,
대구,5.0,4.0,3.0
부산,7.0,1.0,4.0
서울,8.0,0.0,7.0
인천,4.0,3.0,4.0


### 시각화하고 싶엉...

In [94]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

## 이상치 처리

In [165]:
np.random.seed(42)
n = 1000

# 정상적인 데이터 생성
normal_prices = np.random.normal(50000, 15000, 900)  # 정상 가격대
normal_prices = np.round(normal_prices).astype(int)  # 정수 가격

normal_quantities = np.random.poisson(3, 900) + 1   # 정상 수량
normal_quantities = normal_quantities.astype(int)    # 정수 수량

# 이상치 데이터 추가
outlier_prices = np.random.uniform(200000, 500000, 100)  # 이상 고가 상품
outlier_prices = np.round(outlier_prices).astype(int)

outlier_quantities = np.random.uniform(50, 100, 100)      # 이상 대량 주문
outlier_quantities = np.round(outlier_quantities).astype(int)

# 전체 데이터 결합
prices = np.concatenate([normal_prices, outlier_prices])
quantities = np.concatenate([normal_quantities, outlier_quantities])

# DataFrame 생성
data = {
    '주문번호': [f'ORD{i:04d}' for i in range(1, n+1)],
    '가격': prices,
    '수량': quantities,
    '카테고리': np.random.choice(['전자제품', '의류', '가구', '식품', '도서'], n),
    '지역': np.random.choice(['서울', '부산', '인천', '대구', '광주'], n)
}

df = pd.DataFrame(data)
df['총금액'] = df['가격'] * df['수량']

df.head()

,주문번호,가격,수량,카테고리,지역,총금액
0,ORD0001,57451,5,가구,부산,287255
1,ORD0002,47926,5,식품,서울,239630
2,ORD0003,59715,4,식품,부산,238860
3,ORD0004,72845,7,도서,부산,509915
4,ORD0005,46488,3,식품,광주,139464


In [130]:
from scipy import stats

# 이상치 탐지
# 1. Z-score(표준점수)
price_series = df['가격']
price_series.std()  # 전체에 대한 표준편차

# 각 데이터에 대한 표준편차 구하기
# Z-score 직접계산
(price_series - price_series.mean()) / price_series.std()
# 각 항목의 Z-score.(시리즈)

def detect_oulier_zscore(data_series, threshold=3.0):
    from scipy import stats
    
    # scipy 사용
    z_scores = stats.zscore(data_series)
    over_z = np.abs(data_series) > threshold
    return np.abs(z_scores) > threshold

# 이상치 데이터만 보기
# df[over_z].count()  # 40개.

detect_oulier_zscore(df['가격'])



array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [133]:
# 2. IQR(InterQuartile Range - 4분위) 방법
def detect_oulier_iqr(data_series):
    q1 = price_series.quantile(0.25)    # 상위 75%
    price_series.describe()
    q3 = price_series.quantile(0.75)    # 상위 25%
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)

    return (price_series < lower_bound) | (price_series > upper_bound)


df[detect_oulier_iqr(df['가격'])]


,주문번호,가격,수량,카테고리,지역,총금액
209,ORD0210,107791,3,의류,서울,323373
262,ORD0263,1381,4,전자제품,대구,5524
900,ORD0901,498299,52,가구,서울,25911548
901,ORD0902,269819,59,전자제품,서울,15919321
902,ORD0903,216149,70,의류,광주,15130430
...,...,...,...,...,...,...
995,ORD0996,211943,88,식품,부산,18650984
996,ORD0997,282196,91,식품,서울,25679836
997,ORD0998,486695,67,식품,인천,32608565
998,ORD0999,486090,72,도서,광주,34998480


In [143]:
# 3. 백분위수 방법
# 이거는 위랑 다르게 커스텀을 좀 할 수 있음

def detect_oulier_perc(data_series: pd.Series, lower=1, upper=99):
    lower_bound = data_series.quantile(lower / 100) # 하위 1%
    upper_bound = data_series.quantile(upper / 100) # 상위 1%

    return (price_series < lower_bound) | (price_series > upper_bound)

# outlier df
odf = df[detect_oulier_perc(df['가격'], 3.0)]
odf.head()

,주문번호,가격,수량,카테고리,지역,총금액
13,ORD0014,21301,5,식품,서울,106505
14,ORD0015,24126,5,전자제품,서울,120630
37,ORD0038,20605,2,의류,광주,41210
49,ORD0050,23554,4,의류,서울,94216
74,ORD0075,10704,6,도서,부산,64224


### ***글자의 이상치는 어떻게 처리를 해야하나~?~?~?***
> 나중에...

### ***이상치 처리***

In [144]:
df.head()

,주문번호,가격,수량,카테고리,지역,총금액
0,ORD0001,57451,5,가구,부산,287255
1,ORD0002,47926,5,식품,서울,239630
2,ORD0003,59715,4,식품,부산,238860
3,ORD0004,72845,7,도서,부산,509915
4,ORD0005,46488,3,식품,광주,139464


In [154]:
# Data에서 col의 IQR 기준 이상치를 삭제하는 함수
# + 매개변수 정의에 따른 자동완성 이야기    # 써놓으면 좋은 점이 뭔지 알겠네~?
# 반환값의 종류도 알려준다고...? 이거 뭔가 기괴하네.
def remove_outliers_iqr(data: pd.DataFrame, col: str) -> pd.DataFrame:   # 매개변수 설명해주기.
    Q1 = data[col].quantile(0.25)   # 이거 자동완성 왜 안 뜨냐. data 유형이 어떤 건지 말을 안 해줘서 ㅇㅇ
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - (1.5 * IQR)
    upper_bound = Q3 + (1.5 * IQR)
    mask = (data[col] > lower_bound) & (data[col] < upper_bound)
    return data[mask]

df_rm1 = remove_outliers_iqr(df,'가격')
print(len(df), len(df_rm1))
df.describe()
df_rm1.describe() # 이상치를 삭제하면 평균과 중앙값이 맞아가는 모습

1000 898


,가격,수량,총금액
count,898.000000,898.000000,898.000000
mean,50266.181514,3.977728,199222.511136
std,14512.734838,1.689879,104521.213651
min,9547.000000,1.000000,9547.000000
25%,40255.250000,3.000000,122845.500000
50%,50276.500000,4.000000,178333.500000
75%,59727.000000,5.000000,259776.250000
max,96183.000000,10.000000,708800.000000


### 삭제가 아니라 `변환(Transfrom)`를 시켜봅니다.
> 대체(imputation)도 배웁시다

In [ ]:
# 이상치 대체

# A. 평균 대체
## but, 냅다 평균 대체는 좋지 못 함. WHY? 오염된 데이터를 반영한 평균을 넣어서 뭐가 좋음.
### 그러면 이상치 빼고 평균값을 넣어도 됨.


# B. 중앙값 대체(퀀타일 50%가 중앙값임.)
print(len(df))
def replace_outliers_with_median(data: pd.DataFrame, col:str) -> pd.DataFrame:
    """IQR 이상치를 중앙값으로 대체"""
    data_replaced = data.copy()
    # IQR 이상치 탐지
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - (1.5 * IQR)
    upper_bound = Q3 + (1.5 * IQR)
    # 중앙값
    median_val = data[col].median()
    mask = (data[col] < lower_bound) | (data[col] > upper_bound)
    data_replaced.loc[mask, col] = median_val
    return data_replaced

replace_outliers_with_median(df,'가격')
# print(len(df))
df[df['가격'] == df['가격'].median()] # 이건 새로 계산된 median임. 안 나올 수 있음.

1000


/var/folders/tx/t7px7cn503bgk6zlg3m0b4740000gn/T/ipykernel_99166/1342950857.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '52255.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_replaced.loc[mask, col] = median_val


,주문번호,가격,수량,카테고리,지역,총금액


In [172]:
# Winsorization(윈저화): 이상치를 근접값으로 바꾸기.
def wnsrz_outlrs(data:pd.DataFrame, col, lower=5, upper=95):
    lower_bound = data[col].quantile(lower / 100)
    upper_bound = data[col].quantile(upper / 100)
    data_wnsrz = data.copy()
    data_wnsrz[col] = data_wnsrz[col].clip(lower=lower_bound, upper=upper_bound)
    return data_wnsrz
# col을 clip한 상태로 반환하면 값은 시리즈임.

In [174]:
wnsrz_outlrs(df, '가격')

,주문번호,가격,수량,카테고리,지역,총금액
0,ORD0001,57451.0,5,가구,부산,287255
1,ORD0002,47926.0,5,식품,서울,239630
2,ORD0003,59715.0,4,식품,부산,238860
3,ORD0004,72845.0,7,도서,부산,509915
4,ORD0005,46488.0,3,식품,광주,139464
...,...,...,...,...,...,...
995,ORD0996,211943.0,88,식품,부산,18650984
996,ORD0997,282196.0,91,식품,서울,25679836
997,ORD0998,318148.4,67,식품,인천,32608565
998,ORD0999,318148.4,72,도서,광주,34998480
